In [ ]:
import os
import csv
import pandas as pd
import numpy as np
import bisect
from PIL import Image
import shutil
import random
import multiprocessing

num_classes = 2000

In [ ]:
df = pd.read_csv('/home/kylecshan/data/CSVs/train.csv')

In [ ]:
df = df.sort_values('landmark_id')

In [ ]:
#threshold = 100
counts = df.landmark_id.value_counts()
counts = pd.DataFrame(counts)

In [ ]:
#threshold_cts = counts.loc[counts.landmark_id >= threshold, :]
#threshold_cts.columns = ['count']
#threshold_cts
threshold_cts = counts.iloc[:num_classes,:]
threshold_cts.columns = ['count']
threshold_cts.shape

In [ ]:
def mkdir_if_not_exist(path):
    if not os.path.exists(path):
        os.mkdir(path)

In [ ]:
base_dir = '/home/kylecshan/data/images224/train_ms2000_v5/'
imgs_dir = '/home/kylecshan/data/images224/all/'

mkdir_if_not_exist(base_dir)
mkdir_if_not_exist(base_dir + 'train/')
mkdir_if_not_exist(base_dir + 'val/')
mkdir_if_not_exist(base_dir + 'index/')
mkdir_if_not_exist(base_dir + 'query/')

In [ ]:
landmark_list = []
for row in threshold_cts.itertuples():
    landmark, count = row.Index, row.count
    start = df.landmark_id.searchsorted(landmark, side='left')
    end = df.landmark_id.searchsorted(landmark, side='right')
    
    # Randomly shuffle indexes for imgs of this landmark
    np.random.seed(321)
    idx_shuffle = np.random.permutation(range(start, end))
    border = 24 + (count // 6)
    num_to_copy = border + (count//24)
    landmark_list.append((landmark, df.id.iloc[idx_shuffle].values.tolist(), num_to_copy, border))
    

In [ ]:
[x for x in landmark_list if x[0] == 171683]

In [ ]:
def copy_landmark(row):
    landmark, imgnames, num_to_copy, border = row
    num_copied = 0;
    num_seen = 0;
    
    train_dir = base_dir + 'train/' + str(landmark) + '/'
    val_dir = base_dir + 'val/' + str(landmark) + '/'
    index_dir = base_dir + 'index/' + str(landmark) + '/'
    query_dir = base_dir + 'query/' + str(landmark) + '/'
    for d in [train_dir, val_dir, index_dir, query_dir]:
        mkdir_if_not_exist(d)
        
    for imgname in imgnames:
        img_path = imgs_dir + imgname + '.jpg'
        try:
            im = Image.open(img_path)
            w,h = im.size
            if num_copied < 4:
                shutil.copy(img_path, query_dir)
            elif num_copied < 24:
                shutil.copy(img_path, index_dir)
            elif num_copied < border:
                shutil.copy(img_path, train_dir)
            else:
                shutil.copy(img_path, val_dir)
            num_copied += 1
        except:
            pass # couldn't find image, oh well
        if num_copied >= num_to_copy:
            break
        
    print('landmark %d: copied %d out of %d total' % 
          (landmark, num_copied, len(imgnames)))
    return num_copied

In [ ]:
copy_landmark(landmark_list[6])

In [ ]:
pool = multiprocessing.Pool(processes=16)

In [ ]:
pool.map(copy_landmark, landmark_list)

In [ ]:
np.sum(counts.landmark_id < 10)

In [ ]:
len(counts)